# Preprocessing text with Spark NPL - Overview





## 1. Prepare Environment

In [1]:
import os

!apt-get update -qq
!apt-get install -y openjdk-8-jdk-headless -qq

#Install Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
openjdk version "1.8.0_312"
OpenJDK Runtime Environment (build 1.8.0_312-8u312-b07-0ubuntu1~20.04-b07)
OpenJDK 64-Bit Server VM (build 25.312-b07, mixed mode)


In [2]:
#Install Pyspark
! pip install --ignore-installed pyspark==2.4.4

#Install Spark NLP
! pip install --ignore-installed spark-nlp==2.6.2

  Using cached pyspark-2.4.4-py2.py3-none-any.whl
  Using cached py4j-0.10.7-py2.py3-none-any.whl (197 kB)
  Using cached spark_nlp-2.6.2-py2.py3-none-any.whl (128 kB)
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/opt/conda/lib/python3.8/site-packages/com/__init__.py'
Consider using the `--user` option or check the permissions.



## 2. Start Spark Session

In [3]:
import sparknlp
spark = sparknlp.start()

from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

22/04/12 18:40:32 WARN Utils: Your hostname, DSGPU05 resolves to a loopback address: 127.0.1.1; using 10.10.11.64 instead (on interface eno1)
22/04/12 18:40:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.2.1-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/mmccord/.ivy2/cache
The jars for the packages stored in: /home/mmccord/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-befd48c5-918c-4946-89cb-1ead110cd517;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.4.2 in central
	found com.typesafe#config;1.4.1 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lo

## 3. Load and Read Dataset

In this part, I am reading **.csv files** that I generated from the **classics** .txt files.
These .csv files were **generated in python**, exporting the final dataframes we already built to csv.
I am sharing these csv files **in GIT** (https://github.com/mpmccord/FanFicVsClassicLiterature/tree/main/data). 

There are 4 of them:


1.   **classics_clean.csv:** The eight classics with the whole corpus, preprocessed (lower case, remove spaces, etc.)
2.   **classics_raw.csv:**  The eight classics with the whole corpus, raw 
3.   **classics_clean_test.csv:** Subset of cleaned classics. Just two of them with only 200 words of the corpus.
4.   **classics_raw_test.csv:** Subset of raw classics. Just two of them with only 200 words of the corpus.



### 3.1 Read classics_raw_test.csv

For testing porpuses and for short computational times, I am going to use this subset with only two incomplete raw texts

In [6]:
#Generate Spark dataframe from csv file
df_Spark = spark.read \
           .option("header", True) \
           .csv("data/fanfics_raw.csv") #Change path accordingly to yours

df_Spark.show(2, truncate=200)

+-----------------------------------------+-----------------------+-----------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|                                      _c0|                   text|                                                                                                         id|                                                                                                type|
+-----------------------------------------+-----------------------+-----------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|                                        0|              Chapter 1|                                                                                                      

22/04/12 18:41:12 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , text, id, type
 Schema: _c0, text, id, type
Expected: _c0 but found: 
CSV file: file:///home/mmccord/Distributed_Computing/FanFicVsClassicLiterature/data/fanfics_raw.csv


## 4. Create Pipeline

Create pipeline to preprocess the spark dataframe texts.

Each of these classes receive an input column and creates the output column.
At the end of the pipeline, we will have a dataframe with all of the columns that are created on the fly and their results.

The **last column** generated, in this case **token_features** is the one that has all the words after being preprocessed, removing stop words, etc.

In [ ]:
#https://medium.com/spark-nlp/spark-nlp-101-document-assembler-500018f5f6b5
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document") \
    .setCleanupMode("shrink_full") #remove new lines and tabs, plus shrinking spaces and blank lines.

#We dont need this because when preprocessing, the test ends up being one sentence
sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

#https://nlp.johnsnowlabs.com/api/python/reference/autosummary/sparknlp.annotator.Tokenizer.html
token = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

#https://nlp.johnsnowlabs.com/docs/en/annotators
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized") \
    .setLowercase(True) \
    .setCleanupPatterns(["""[^A-Za-z]"""]) # remove punctuations and alphanumeric chars

#Stop words used by Spark NLP: http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words
#https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/b2eb08610dd49d5b15077cc499a94b4ec1e8b861/jupyter/annotation/english/stop-words/StopWordsCleaner.ipynb#scrollTo=1-eGocORg2ml
stop_words = StopWordsCleaner.pretrained('stopwords_en', 'en')\
    .setInputCols(["normalized"]) \
    .setOutputCol("cleanTokens") \
    .setCaseSensitive(False)

#https://nlp.johnsnowlabs.com/api/com/johnsnowlabs/nlp/annotators/LemmatizerModel.html
lemmatizer = LemmatizerModel.pretrained() \
         .setInputCols(["cleanTokens"]) \
         .setOutputCol("lemma")

finisher = Finisher() \
    .setInputCols(["lemma"]) \
    .setOutputCols(["token_features"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)

In [ ]:
nlp_pipeline_lr = Pipeline(
        stages=[document, 
                sentence,
                token,
                normalizer,
                stop_words, 
                lemmatizer, 
                finisher])

## 5. Apply Pipeline to Spark Dataframe

In [ ]:
processed_text = nlp_pipeline_lr.fit(df_Spark).transform(df_Spark)

In [ ]:
processed_text.show(truncate=200) #Show results

In [ ]:
#Show last column, the one that has the final result
processed_text.select("token_features").show(truncate=200) 